# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

In [756]:
# For degree celcius: (\xb0C)

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


Load data from the config file

In [757]:
import yaml
with open("config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)
knmi = config['data_knmi']

In [758]:
import pandas as pd
import numpy as np
df = pd.read_csv(knmi, comment="#", header=None, dtype={2:str, 3:str ,4:str ,5:str , 6:str })
df.columns = ['station','date','avg temp', 'min temp', 'max temp','sunshine','rainfall','sun of rainfall']

There is a description of the data added with comments, so filter by: comment="#" 

Directly trying to make it into a dataframe gives a Dtypewarning for column 2,3,4,5 and 6, so specify the dtype as string. (There is string in the columns and those can't be directly converted to floats or ints)


The header is in the commented section, so add a new header with readable names

In [759]:
df

,station,date,avg temp,min temp,max temp,sunshine,rainfall,sun of rainfall
0,209,20010130,,,,,,NaN
1,209,20010131,,,,,,NaN
2,209,20010201,,,,,,NaN
3,209,20010202,,,,,,NaN
4,209,20010203,,,,,,NaN
...,...,...,...,...,...,...,...,...
331311,391,20181227,12,-18,47,28,0,0.0
331312,391,20181228,7,-29,30,23,0,0.0
331313,391,20181229,59,25,92,0,15,5.0
331314,391,20181230,78,52,87,3,42,17.0


Select station 270 leeuwarden

In [760]:
df_leeuwarden = df.loc[df['station'] == 270]

In [761]:
df_leeuwarden

,station,date,avg temp,min temp,max temp,sunshine,rainfall,sun of rainfall
97641,270,20000101,42,-4,79,49,15,11.0
97642,270,20000102,55,33,74,12,0,-1.0
97643,270,20000103,74,49,89,0,124,172.0
97644,270,20000104,46,22,75,4,13,11.0
97645,270,20000105,41,14,56,56,0,0.0
...,...,...,...,...,...,...,...,...
104576,270,20181227,57,53,62,0,9,2.0
104577,270,20181228,71,58,81,0,0,0.0
104578,270,20181229,85,69,102,0,14,18.0
104579,270,20181230,80,68,90,0,14,5.0


Select only the columns needed: station, avg temp, min temp and max temp

In [762]:
df_leeuwarden = df_leeuwarden[df_leeuwarden.columns[0:5]]

In [763]:
df_leeuwarden.head()

,station,date,avg temp,min temp,max temp
97641,270,20000101,42,-4,79
97642,270,20000102,55,33,74
97643,270,20000103,74,49,89
97644,270,20000104,46,22,75
97645,270,20000105,41,14,56


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [764]:
#replace cells with spaces to NaN
#change data formats
#change temperatures to celcius degrees
#remove leap year

From the gitbook: Columns might contain a string indicating missing values and therefore the entire column is set to a string datatype.

Replace cells with spaces to NaN and typecast to the right format

In [765]:
df_leeuwarden.replace(r'^\s*$', np.nan, regex=True) 
df_leeuwarden['date'] = pd.to_datetime(df_leeuwarden['date'].astype(str), format='%Y%m%d')
df_leeuwarden['avg temp'] = pd.to_numeric(df_leeuwarden['avg temp'])
df_leeuwarden['min temp'] = pd.to_numeric(df_leeuwarden['min temp'])
df_leeuwarden['max temp'] = pd.to_numeric(df_leeuwarden['max temp'])

/tmp/ipykernel_3319140/2410433025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden['date'] = pd.to_datetime(df_leeuwarden['date'].astype(str), format='%Y%m%d')
/tmp/ipykernel_3319140/2410433025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden['avg temp'] = pd.to_numeric(df_leeuwarden['avg temp'])
/tmp/ipykernel_3319140/2410433025.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [766]:
df_leeuwarden.dtypes

station              int64
date        datetime64[ns]
avg temp             int64
min temp             int64
max temp             int64
dtype: object

In the file is stated that the temperatures are in 0.1 degrees Celsius, however in the assignment it's mentioned it's centidegrees. Looking at the values it seems to be indeed centidegrees, so the values should be divided by 10

In [767]:
df_leeuwarden['avg temp'] = df_leeuwarden['avg temp'].div(10)
df_leeuwarden['min temp'] = df_leeuwarden['min temp'].div(10)
df_leeuwarden['max temp'] = df_leeuwarden['max temp'].div(10)

/tmp/ipykernel_3319140/2949216423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden['avg temp'] = df_leeuwarden['avg temp'].div(10)
/tmp/ipykernel_3319140/2949216423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden['min temp'] = df_leeuwarden['min temp'].div(10)
/tmp/ipykernel_3319140/2949216423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [768]:
df_leeuwarden

,station,date,avg temp,min temp,max temp
97641,270,2000-01-01,4.2,-0.4,7.9
97642,270,2000-01-02,5.5,3.3,7.4
97643,270,2000-01-03,7.4,4.9,8.9
97644,270,2000-01-04,4.6,2.2,7.5
97645,270,2000-01-05,4.1,1.4,5.6
...,...,...,...,...,...
104576,270,2018-12-27,5.7,5.3,6.2
104577,270,2018-12-28,7.1,5.8,8.1
104578,270,2018-12-29,8.5,6.9,10.2
104579,270,2018-12-30,8.0,6.8,9.0


Remove leap years: month == 2 & day == 29
Every year that contains 29 day in februari is a leap year and should be removed
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.is_leap_year.html

In [769]:
df_leeuwarden['leapyear'] = df_leeuwarden.date.dt.is_leap_year
df_leeuwarden


/tmp/ipykernel_3319140/3019594865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden['leapyear'] = df_leeuwarden.date.dt.is_leap_year


,station,date,avg temp,min temp,max temp,leapyear
97641,270,2000-01-01,4.2,-0.4,7.9,True
97642,270,2000-01-02,5.5,3.3,7.4,True
97643,270,2000-01-03,7.4,4.9,8.9,True
97644,270,2000-01-04,4.6,2.2,7.5,True
97645,270,2000-01-05,4.1,1.4,5.6,True
...,...,...,...,...,...,...
104576,270,2018-12-27,5.7,5.3,6.2,False
104577,270,2018-12-28,7.1,5.8,8.1,False
104578,270,2018-12-29,8.5,6.9,10.2,False
104579,270,2018-12-30,8.0,6.8,9.0,False


In [770]:
df_leeuwarden = df_leeuwarden.loc[df_leeuwarden['leapyear'] == False]
df_leeuwarden.drop('leapyear', axis=1, inplace = True)

/tmp/ipykernel_3319140/2297319390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leeuwarden.drop('leapyear', axis=1, inplace = True)


In [771]:
df_leeuwarden

,station,date,avg temp,min temp,max temp
98007,270,2001-01-01,2.1,0.4,3.8
98008,270,2001-01-02,6.3,3.7,8.5
98009,270,2001-01-03,5.3,2.6,7.9
98010,270,2001-01-04,6.5,4.6,7.9
98011,270,2001-01-05,6.6,5.6,8.3
...,...,...,...,...,...
104576,270,2018-12-27,5.7,5.3,6.2
104577,270,2018-12-28,7.1,5.8,8.1
104578,270,2018-12-29,8.5,6.9,10.2
104579,270,2018-12-30,8.0,6.8,9.0


In [772]:
df_leeuwarden.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 98007 to 104580
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   station   5110 non-null   int64         
 1   date      5110 non-null   datetime64[ns]
 2   avg temp  5110 non-null   float64       
 3   min temp  5110 non-null   float64       
 4   max temp  5110 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 239.5 KB


For the Expected outcome the index is Datetime, so change the index accordingly. Automatically drops the date column. (Keeping the date column seems redundant)

In [773]:
df_leeuwarden.set_index(['date'], drop=False, inplace = True)
df_leeuwarden.sort_index(inplace=True)
df_leeuwarden

,station,date,avg temp,min temp,max temp
date,,,,,
2001-01-01,270,2001-01-01,2.1,0.4,3.8
2001-01-02,270,2001-01-02,6.3,3.7,8.5
2001-01-03,270,2001-01-03,5.3,2.6,7.9
2001-01-04,270,2001-01-04,6.5,4.6,7.9
2001-01-05,270,2001-01-05,6.6,5.6,8.3
...,...,...,...,...,...
2018-12-27,270,2018-12-27,5.7,5.3,6.2
2018-12-28,270,2018-12-28,7.1,5.8,8.1
2018-12-29,270,2018-12-29,8.5,6.9,10.2


In [774]:
#Test your outcome
#write code to check if you have done the above
df_leeuwarden.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5110 entries, 2001-01-01 to 2018-12-31
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   station   5110 non-null   int64         
 1   date      5110 non-null   datetime64[ns]
 2   avg temp  5110 non-null   float64       
 3   min temp  5110 non-null   float64       
 4   max temp  5110 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 239.5 KB


Looking into the file, the first date registerd is 2001-01-01 and not 1951-01-01. So it's correct that the index starts there

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [775]:
#df_leeuwarden.date.dt.month

In [776]:
#df_leeuwarden.date.dt.day

In [777]:
df = df_leeuwarden

In [778]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5110 entries, 2001-01-01 to 2018-12-31
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   station   5110 non-null   int64         
 1   date      5110 non-null   datetime64[ns]
 2   avg temp  5110 non-null   float64       
 3   min temp  5110 non-null   float64       
 4   max temp  5110 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 239.5 KB


In [779]:
# If you keep the date column you can groupby via that column instead of the index:
# df_leeuwarden.groupby([(df_leeuwarden.date.dt.month.rename('month')), (df_leeuwarden.date.dt.day.rename('day'))])['min temp'].min()

In [780]:
# Function to group by the datetime index and group by month and day for the minimum value of that day
def month_day_min(df_multipleyears):
    df_grouped = df.groupby([(df.index.month.rename('month')), (df.index.day.rename('day'))])['min temp'].min()
    
    print(df_grouped)
month_day(df)

month  day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -6.7
       5      -6.2
              ... 
12     27     -6.0
       28     -7.4
       29     -7.3
       30     -6.7
       31     -5.1
Name: min temp, Length: 365, dtype: float64


In [781]:
#Test your code
def test_reformed(df):
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day_min(df)

test_reformed(df)

month  day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -6.7
       5      -6.2
              ... 
12     27     -6.0
       28     -7.4
       29     -7.3
       30     -6.7
       31     -5.1
Name: min temp, Length: 365, dtype: float64


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [ ]:
#Signal analysis will be discussed in januari 2023

In [ ]:
# https://corporatefinanceinstitute.com/resources/business-intelligence/data-smoothing/
# numpy.convolve
# statsmodels.kernel_regression
# Averaging Methods
# Exponential Smoothing Methods
# weighted least-squares linear regression (with moving temporal window to calculate a regression line.) 

#### Savgol filter

-  scipy.signal.savgol_filter()
-  Savitzky-Golay filter is a digital filter that uses data points for smoothing the graph. 
-  It uses the method of least squares that creates a small window and applies a polynomial on the data of that window, and then uses that polynomial for assuming the center point of the particular window. 
-  https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html


In [864]:
from scipy.signal import savgol_filter
x = df_2018['avg temp']
sav = savgol_filter(x, 5, 2, mode='nearest')

#### Rolling mean: moving averages

-  Creating a rolling average allows you to “smooth” out small fluctuations in datasets,while gaining insight into trends.
-  It allows for smoothing out data with high degrees of fluctuation.
-  https://datagy.io/rolling-average-pandas/
-  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html


---

In [866]:
# f.e.
df['min temp'].rolling(5).mean()

date
2001-01-01    0.4
2001-01-02    3.7
2001-01-03    2.6
2001-01-04    4.6
2001-01-05    5.6
             ... 
2018-12-27    5.3
2018-12-28    5.8
2018-12-29    6.9
2018-12-30    6.8
2018-12-31    8.2
Name: min temp, Length: 5110, dtype: float64

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

To get the ultimate min and max values, again groupby month and day for the min, max and average temperature

For plotting you don't want a multi-index and the days of the year go from 1 to 365, so reset the index to go from 1 to 365

In [783]:
df_ult_vals = df.groupby([(df.index.month.rename('month')), (df.index.day.rename('day'))]).agg({'min temp': 'min', 'max temp':'max', 'avg temp':'mean'})
df_ult_vals = df_ult_vals.reset_index()
df_ult_vals.index = range(1, len(df_ult_vals)+1)

In [784]:
df_ult_vals

,month,day,min temp,max temp,avg temp
1,1,1,-5.8,10.1,3.700000
2,1,2,-7.5,10.2,4.128571
3,1,3,-12.6,10.7,3.742857
4,1,4,-6.7,9.8,3.764286
5,1,5,-6.2,9.4,2.714286
...,...,...,...,...,...
361,12,27,-6.0,11.7,4.514286
362,12,28,-7.4,10.9,4.450000
363,12,29,-7.3,10.2,3.400000
364,12,30,-6.7,11.1,3.685714


Select te year 2018

In [785]:
df_2018 = df[df.index.year == 2018]
df_2018

,station,date,avg temp,min temp,max temp
date,,,,,
2018-01-01,270,2018-01-01,6.0,4.0,7.9
2018-01-02,270,2018-01-02,5.6,3.1,7.5
2018-01-03,270,2018-01-03,7.5,5.3,9.2
2018-01-04,270,2018-01-04,7.3,5.8,9.1
2018-01-05,270,2018-01-05,6.0,4.0,7.6
...,...,...,...,...,...
2018-12-27,270,2018-12-27,5.7,5.3,6.2
2018-12-28,270,2018-12-28,7.1,5.8,8.1
2018-12-29,270,2018-12-29,8.5,6.9,10.2


For the plot make sure that the index of the two dataframes are the same

In [786]:
df_2018.index = range(1,len(df_2018)+1)
df_2018

,station,date,avg temp,min temp,max temp
1,270,2018-01-01,6.0,4.0,7.9
2,270,2018-01-02,5.6,3.1,7.5
3,270,2018-01-03,7.5,5.3,9.2
4,270,2018-01-04,7.3,5.8,9.1
5,270,2018-01-05,6.0,4.0,7.6
...,...,...,...,...,...
361,270,2018-12-27,5.7,5.3,6.2
362,270,2018-12-28,7.1,5.8,8.1
363,270,2018-12-29,8.5,6.9,10.2
364,270,2018-12-30,8.0,6.8,9.0


In [815]:
df_2018['smooth'] = df_2018['avg temp'].rolling(5).mean()

/tmp/ipykernel_3319140/2121304760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['smooth'] = df_2018['avg temp'].rolling(5).mean()


In [816]:
df_2018

,station,date,avg temp,min temp,max temp,smooth
1,270,2018-01-01,6.0,4.0,7.9,NaN
2,270,2018-01-02,5.6,3.1,7.5,NaN
3,270,2018-01-03,7.5,5.3,9.2,NaN
4,270,2018-01-04,7.3,5.8,9.1,NaN
5,270,2018-01-05,6.0,4.0,7.6,6.48
...,...,...,...,...,...,...
361,270,2018-12-27,5.7,5.3,6.2,6.06
362,270,2018-12-28,7.1,5.8,8.1,6.18
363,270,2018-12-29,8.5,6.9,10.2,6.94
364,270,2018-12-30,8.0,6.8,9.0,7.28


In [789]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Legend, Band, ColumnDataSource, LinearAxis, Range1d
import panel as pn
from bokeh.models.tickers import FixedTicker

In [867]:
output_notebook()
pn.extension()
# Add title and axis labels
p = figure(title='Weather data of 2018', width=900, height=600, x_range=(1,365), y_range=(-20,35))
p.xaxis.axis_label = 'Day of the year'
p.yaxis.axis_label = 'Temperature (\xb0C)'

# Select data needed from dataframe for the bandsource
data = {'min': df_ult_vals['min temp'],
        'max': df_ult_vals['max temp'],
       'mean': df_ult_vals['avg temp'],
       'days':df_ult_vals.index}

bandsource = ColumnDataSource(data=data)

# Create band based on the minimum and maximum temperatures
band = Band(base="days", lower="min", upper="max", source=bandsource,
            fill_alpha=0.3, fill_color="gray", line_color="black")
p.add_layout(band)

# Create plots
r = p.line(x=df_2018.index, y=df_2018['smooth'], line_width=1, color='blue')
t = p.line(x=df_2018.index, y=df_2018['avg temp'], line_width=1, color='red')
d = p.circle(x=df_2018.index, y=df_2018['avg temp'], color='black', size=1.5)
s = p.circle(x=df_2018.index, y=df_2018['smooth'], color='#53777a', size=2)
sv = p.line(x=df_2018.index, y=sav, color='black')


# Legend layout
legend = Legend(items=[("mean temperature per day", [d]),("mean temperature",[t]),
                      ('rolling mean',[r]),('rolling mean points',[s]),('savgol filter',[sv])], location="center")
p.add_layout(legend, 'below')
p.legend.click_policy='hide'
p.legend.border_line_width = 3
p.legend.border_line_color = "navy"
p.legend.border_line_alpha = 0.1

# Grid lay-out
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None

# Add second axis for months
p.extra_x_ranges = {"month": Range1d(start = 0, end = 12)}
p.add_layout(LinearAxis(x_range_name = "month"), 'above')
p.xaxis[0].ticker = FixedTicker(ticks=[1,2,3,4,5,6,7,8,9,10,11,12])
p.xaxis[0].major_label_overrides = {1: 'January', 2: 'February', 3:'March', 4:'April', 5:'May',
                                 6:'June', 7:'July', 8:'August', 9:'September', 10:'October',
                                 11:'November', 12:'December'}

# Haven't fount out how to color band grid for second axis

show(p)

Loading BokehJS ...

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather